In [1]:
import numpy as np
import pandas as pd
import os
import time

In [2]:
# 创建经纬度的网格码对照表(360x720)函数
def cread_grids_table(m,n):

    grids_table = np.array(list(range(1, m*n+1))).reshape(m,n)
    grids_table = pd.DataFrame(grids_table, columns=range(1,n+1), index=range(1,m+1))
    return(grids_table)

In [3]:
# 查找经纬度对应的网格码
def search_grids_num(lat,lon,grids_table):
    
    m = int((90+lat)*2+1)
    n = int((180+lon)*2+1)

    return(grids_table.loc[m,n])

In [4]:
# 读取数据
def load_data(file_name, sheetname=None):
    # 获取文件后缀（区分 .csv 和 .xlsx）
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_name)
    elif file_name.endswith(('.xlsx', '.xls')):  # 兼容 .xls 格式
        df = pd.read_excel(file_name, sheet_name=sheetname)
    else:
        # 非支持格式，可抛异常或返回提示
        raise ValueError(f"不支持的文件格式：{file_name}，仅支持 .csv、.xlsx、.xls")
    return df

In [5]:
# 按照网格码表格生成df数据集的网格化结果
def clear_data(df, grids_table):
    
    grids_num = []
    df_result = pd.DataFrame()
    
    for k in df.index:
        grids_num.append(search_grids_num(df['lat'].values[k],df['lon'].values[k], grids_table)) #计算经纬度所属的网格码
    
    df['grid_num'] = grids_num
    
    # 按 grid_num 分组，对 clay、BD、SOC、pH、CEC 列求平均值
    df_result = df.groupby('grid_num')[df.columns[2:-1].tolist()].mean().round(2)
    df_result = df_result.reset_index()
    
    return(df_result)


In [6]:
if __name__ == "__main__":

    # 记录开始时间
    start_time = time.time()  # 返回当前时间的时间戳（秒）
    
    # 创建网格码对照表实例
    m = int(180/0.5)
    n = int(360/0.5)
    
    grids_table = cread_grids_table(m,n)
    
    # 获取所有的经纬度数据
    filename = './全球人工氮肥施用量(2018)'
    
    # 读取数据
    df = load_data(filename+'.csv')
    
    # 计算网格化结果
    df_result = clear_data(df, grids_table)
    
    # 存储结果
    df_result.to_csv(filename+'(grids).csv', index=False)

    # 记录结束时间
    end_time = time.time()
    
    # 计算耗时
    elapsed_time = end_time - start_time
    print(f"耗时: {elapsed_time/60:.2f} 分")

耗时: 0.22 分
